In [1]:
import nltk # using textBlob and tensoflow (CNN and LSTM )
import pprint
import tensorflow as tf
import pandas as pd
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
# some source for this dic 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mayti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data_train = pd.read_csv('C:/Project/movie_reviews/movie_reviews.csv')
data_test = pd.read_csv('C:/Project/test_data.csv')

In [3]:
data_train['label'].value_counts()

label
0    20049
1    19951
Name: count, dtype: int64

In [4]:
data_train

,text,label
0,If you havent seen this movie than you need to...,1
1,but Cinderella gets my vote not only for the w...,0
2,This movie is pretty cheesy but I do give it c...,1
3,I have not seen a Van Damme flick for a while ...,1
4,This is a sleeper It defines Nicholas Cage The...,1
...,...,...
39995,This movie is inspiring to anyone who is or ha...,1
39996,This film hits the heart with a reality like n...,1
39997,East Side Story is a documentary of musical co...,1
39998,And a selfadmitted one to boot At one point th...,0


In [5]:
sentiment_label = data_train['label'].values
sentiment_label


array([1, 0, 1, ..., 1, 0, 0], dtype=int64)

In [6]:
tweet = data_train.text.str.lower().values

In [7]:
tweet

array(['if you havent seen this movie than you need to it rocks and you have to watch it it is so funny and will make you laugh your guts out so you have to watch it and i saw it about a billion and a half times and still think it is funny so you have to yes i have memorized the whole movie and could quote it to you from start to finish you must see this move it is also cute because it is half a chick flick if you dont watch it then you are really missing outthis movie even has cute guys in it and that is always a bonus so in summary watch the movie now and trust me you will not be making a mistake did i mention the music is good too so you should like it if you enjoy music this is a movie that they rated correctly and it will work for anyone',
       'but cinderella gets my vote not only for the worst of disneys princess movies but for the worst movie the company made during walts lifetime the music is genuinely pretty and the story deserves to be called classic what fails in this mov

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)

tokenizer.fit_on_texts(tweet)


In [9]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [11]:
arr = np.array(padded_sequence)
count_zeros = np.count_nonzero(arr == 0, axis=1)

In [12]:
print(count_zeros.mean())

62.8742


In [13]:
padded_sequence

array([[  0,   0,   0, ..., 158,  14, 240],
       [137,   7, 332, ...,   4,  10, 188],
       [  0,   0,   0, ...,  30,  78, 828],
       ...,
       [  0,   0,   0, ..., 523,  43,   4],
       [ 49,   1, 490, ...,  27, 700, 513],
       [  0,   0,   0, ..., 143,  10,  16]])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size =5000

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           160000    
                                                                 
 spatial_dropout1d (Spatial  (None, 200, 32)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 176651 (690.04 KB)
Trainable params: 176651 (690.04 KB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [15]:
history = model.fit(padded_sequence,sentiment_label,validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5


1000/1000 [==============================] - 48s 46ms/step - loss: 0.4654 - accuracy: 0.7789 - val_loss: 0.4294 - val_accuracy: 0.7921
Epoch 2/5
1000/1000 [==============================] - 65s 65ms/step - loss: 0.3696 - accuracy: 0.8422 - val_loss: 0.3453 - val_accuracy: 0.8564
Epoch 3/5
1000/1000 [==============================] - 97s 97ms/step - loss: 0.3629 - accuracy: 0.8452 - val_loss: 0.3371 - val_accuracy: 0.8550
Epoch 4/5
1000/1000 [==============================] - 58s 58ms/step - loss: 0.3490 - accuracy: 0.8531 - val_loss: 0.3316 - val_accuracy: 0.8593
Epoch 5/5
1000/1000 [==============================] - 49s 49ms/step - loss: 0.3405 - accuracy: 0.8545 - val_loss: 0.3708 - val_accuracy: 0.8575


In [16]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", abs(sentiment_label[prediction] - 1))
    return abs(sentiment_label[prediction] - 1)


test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)

test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

1/1 [==============================] - 0s 175ms/step
Predicted label:  1
1/1 [==============================] - 0s 23ms/step
Predicted label:  0


0

In [17]:
data_test.head(5)

,Id,text
0,0,What can possibly said about this movie other ...
1,1,I dont care how many bad reviews purple rain g...
2,2,Ken Russell directed this weird Not very ero...
3,3,This is a great movie from the lost age of rea...
4,4,I have a problem with the movie snobs who cons...


In [18]:
data_test['Predicted_Sentiment'] = data_test['text'].str.lower().apply(predict_sentiment)

1/1 [==============================] - 0s 21ms/step
Predicted label:  0
1/1 [==============================] - 0s 21ms/step
Predicted label:  1
1/1 [==============================] - 0s 21ms/step
Predicted label:  0
1/1 [==============================] - 0s 20ms/step
Predicted label:  1
1/1 [==============================] - 0s 24ms/step
Predicted label:  1
1/1 [==============================] - 0s 21ms/step
Predicted label:  1
1/1 [==============================] - 0s 21ms/step
Predicted label:  1
1/1 [==============================] - 0s 20ms/step
Predicted label:  1
1/1 [==============================] - 0s 26ms/step
Predicted label:  1
1/1 [==============================] - 0s 21ms/step
Predicted label:  1
1/1 [==============================] - 0s 27ms/step
Predicted label:  0
1/1 [==============================] - 0s 21ms/step
Predicted label:  0
1/1 [==============================] - 0s 23ms/step
Predicted label:  0
1/1 [==============================] - 0s 24ms/step
Predicted la

In [19]:
submission = data_test.copy()
submission = submission.drop(['text'], axis=1)
submission

,Id,Predicted_Sentiment
0,0,0
1,1,1
2,2,0
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,0
9997,9997,1
9998,9998,0


In [20]:
submission.to_csv('submission3.csv', header=['Id', 'Category'], index=None)